### jupyter-notebook configs  
- kernel: project_gilead_chen (Python 3.10.0)

### Setup
Add 'library_annotation/scripts' to search path

In [2]:
import sys
sys.path.append(
    "/home/woodformation/SSD3/CCC/project_Gilead/library_annotation/scripts"
)

'20220509'

### Generate Sanger sequencing requests from Primer3 outputs (20220503)
GENOMICS Economical Oligo Order Form  

In [18]:
from pathlib import Path
from typing import List
import shutil
import re
from time import strftime, gmtime

# date
date = strftime("%Y%m%d", gmtime())

# Input paths
p3out = Path("outputs/primer3_caller/")
p3config = Path("configs/primer3.config")

# Output paths
outdir = Path(f"SEQUENCING_REQUESTS/{date}/")

# Output children paths
archive = outdir / "archive" 
p3out_arch = archive / "primer3_output"
out_oligo = outdir / f"{date}_oligo_order.csv"
out_seq = outdir / f"{date}_sequencing_order.csv"

# out_oligo header
headers_oligo = [
    "Oligo Title",
    "Sequences(5'to 3')",
    "Quantity(O.D.)",
    "Quality",
    "5' modification",
    "3' modification",
    "Internal",
    "Notes"
]
# out_seq header
headers_seq = [
   "Sample name",
   "Primer name"
]

# make/touch all output folders/files
outdir.mkdir(parents=True, exist_ok=True)
archive.mkdir(parents=True, exist_ok=True)
p3out_arch.mkdir(parents=True, exist_ok=True)

# open out_oligo
with open(out_oligo, "w") as handle:
    handle.write(",".join(headers_oligo) + "\n")
# open out_seq
with open(out_seq, "w") as handle:
    handle.write(",".join(headers_seq) + "\n")


# archiving
# primer3 output to p3out_arch
for child in p3out.iterdir():
    shutil.copy(child, p3out_arch)
# primer3 config to archive
shutil.copy(p3config, archive)

# work with the archived files
p3out = p3out_arch
# function `next_chr`
def next_chr(c:str):
    """returns the next character of `c`"""
    return chr(ord(c) + 1)

# function `prev_chr`
def prev_chr(c:str):
    return chr(ord(c) - 1)

# function `p3parser`
def p3parser(file:str) -> List[dict]:
    with open(file, "r") as handle:
        records = []
        record = {}
        for line in handle:
            line = line.strip()
            if line == "=":
                records.append(record)
                record = {}
            else:
                k, v = line.split("=")
                record[k] = v
    return records

# patterns
prog = re.compile(
    "(?P<sample>[0-9][0-9]_[A-Z][0-9][0-9])[|]?_(?P<pname>AD[A-Z])"
)
# global variables
_QUANTITY = "-"
_QUALITY = "-"
_5MODIF = "-"
_3MODIF = "-"
_INTERNAL = "-"


# record primer sequences to output.csv
flist = [i for i in p3out.iterdir()]
flist.sort()
for fpath in flist:
    if not "primer3" in fpath.name:
        continue
    left = right = None
    p3records = p3parser(fpath)
    try:
        left, right = p3records
    except ValueError:
        if "PRIMER_LEFT_0" in p3records[0].keys():
            left = p3records[0]
        if "PRIMER_RIGHT_0" in p3records[0].keys():
            right = p3records[0]
    
    if left:
        # get the last primer name 
        seqid =left["SEQUENCE_ID"]
        sample, pname = prog.findall(seqid)[-1]
        # output
        oligo_title = sample + "_AD" + next_chr(pname[-1])
        pseq = left["PRIMER_LEFT_0_SEQUENCE"]
        notes = f"For sample {sample}"
        foo = [
            oligo_title,
            pseq,
            _QUANTITY,
            _QUALITY,
            _5MODIF,
            _3MODIF,
            _INTERNAL,
            notes
        ]
        with open(out_oligo, "a") as handle:
            handle.write(",".join(foo) + "\n")
        with open(out_seq, "a") as handle:
            handle.write(f"{sample},AD{next_chr(pname[-1])}\n")
    if right:
        # get the last primer name
        seqid = right["SEQUENCE_ID"]
        sample, pname = prog.findall(seqid)[0] 
        # output
        oligo_title = sample + "_AD" + prev_chr(pname[-1])
        pseq = right["PRIMER_RIGHT_0_SEQUENCE"]
        notes = f"For sample {sample}"
        foo = [
            oligo_title,
            pseq,
            _QUANTITY,
            _QUALITY,
            _5MODIF,
            _3MODIF,
            _INTERNAL,
            notes
        ]
        with open(out_oligo, "a") as handle:
            handle.write(",".join(foo) + "\n")        
        with open(out_seq, "a") as handle:
            handle.write(f"{sample},AD{prev_chr(pname[-1])}\n")


### Extract boundary sequences (20220415)  
Search and record potential boundary sequences from: *'outputs/samtools/filtered/raw_fastq.filtered.sam'*  


In [6]:
from Align.AlignIO import AlignIO
from Bio import SeqIO
import re

# max len extracted
max_len = 20

# input file
fin = "outputs/samtools/filtered/raw_fastq.filtered.sam"

# output boundary fasta
front_bound = f"test/test_clustalo/front_bound_{max_len}.fa"
back_bound = f"test/test_clustalo/back_bound_{max_len}.fa"
open(front_bound, "w").close()
open(back_bound, "w").close()
handle_front = open(front_bound, "a")
handle_back = open(back_bound, "a")

# qname parser
pattern = "(?P<sample>[0-9][0-9]_[A-Z][0-9][0-9])" + \
    "_" + "(?P<suffix>.*)"
prog = re.compile(pattern)


for align in AlignIO(fin).AlignGenerator():
    # skip unmapped record
    if align.flag == 4:
        continue
    
    # get cigars and positions
    clip = align.get_softclip()

    # match qname to pattern
    result = prog.match(align.qname)
    # if "ADF"
    if result.group("suffix") == "ADF":
        # if front clipped
        if clip[0] > 0:
            start = max(0, clip[0] - max_len)
            end = clip[0]
            handle_front.write(
                ">%s\n%s\n" % (
                    result.group("sample"),
                    align.seq[start:end]
                    )
                )
        continue
    # if "ADR"
    if result.group("suffix") == "ADR":
        # if end clipped
        if clip[1]:
            start = -clip[1]
            end = min(-1, -clip[1] + max_len)
            handle_back.write(
                ">%s\n%s\n" % (
                    result.group("sample"),
                    align.seq[start:end]
                    )
            )
        continue

handle_front.close()
handle_back.close()

### Multiple alignment using Clustal Omega

In [8]:
import subprocess

# where is clustalo
clustalo = "./tools/clustalo"

for max_len in [20]:
    for direction in ["front", "back"]:
        # params
        input_file = f"./test/test_clustalo/{direction}_bound_{max_len}.fa"
        output_file = input_file.replace(".fa", ".aln")
        outfmt = "clustal"
        wrap = "200"
        
        # initial cmdline
        cmdline = []
        
        cmdline.append(clustalo)
        if input_file:
            cmdline.append("-i")
            cmdline.append(input_file)
        if outfmt:
            cmdline.append(f"-outfmt={outfmt}")
            # cmdline.append(outfmt)
        if output_file:
            cmdline.append("-o")
            cmdline.append(output_file)
        if wrap:
            cmdline.append("--wrap")
            cmdline.append(wrap)

        subprocess.run(cmdline, capture_output=True)

### Sanger sequence primer design using Primer3 (20220420)
The following criteria are considered most critical in sequencing primer design:
- Primer length should be in the range of 18 and 24 bases.
- The primer should have a GC content of about 45-55%.
- The primers should have a GC-lock (or GC "clamp") on the 3' end (i.e. the last 1 or 2 nucleotides should be a G or C residue).
- The primer should have a melting temperature (Tm) greater than 50°C but less than 65°C.
- The primer should not include homopolymeric runs of more than 4-5 nucleotides.
- Avoid primers with secondary structures or the potential to self-hybridize.
- Avoid designing primer upstream of homopolymeric or heteropolymeric regions (A, C, G or T repeats).
- Check primer for specificity in annealing to template (= lack of secondary priming site).
- Primer should be located at least 50-60 bases upstream of your sequence of interest.

https://dnacore.mgh.harvard.edu/new-cgi-bin/site/pages/sequencing_pages/primer_design.jsp

#### Testing Primer3

Primer3 general settings

In [29]:
# primer siz (14-25, 20)
PRIMER_MIN_SIZE: int = 14
PRIMER_MAX_SIZE: int = 25
PRIMER_OPT_SIZE: int = 20
# primer GC-content (40-50, 50)
PRIMER_MIN_GC: float = 40.0
PRIMER_MAX_GC: float = 50.0
PRIMER_OPT_GC_PERCENT: float = 50.0
# primer melting temperature (Celsius) (40-55, 50)
PRIMER_MIN_TM: float = 40
PRIMER_MAX_TM: float = 55
PRIMER_OPT_TM: float = 50
# the maximum number of primer to return
PRIMER_NUM_RETURN: int = 1
# max Ns accepted
PRIMER_MAX_NS_ACCEPTED: int = 0
# 
PRIMER_EXPLAIN_FLAG: bool = 1

# param dict
GENERAL = {
    "PRIMER_MIN_SIZE": PRIMER_MIN_SIZE,
    "PRIMER_MAX_SIZE": PRIMER_MAX_SIZE,
    "PRIMER_OPT_SIZE": PRIMER_OPT_SIZE,
    "PRIMER_MIN_GC": PRIMER_MIN_GC,
    "PRIMER_MAX_GC": PRIMER_MAX_GC,
    "PRIMER_OPT_GC_PERCENT": PRIMER_OPT_GC_PERCENT,
    "PRIMER_MIN_TM": PRIMER_MIN_TM,
    "PRIMER_MAX_TM": PRIMER_MAX_TM,
    "PRIMER_OPT_TM": PRIMER_OPT_TM,
    "PRIMER_NUM_RETURN": PRIMER_NUM_RETURN,
    "PRIMER_MAX_NS_ACCEPTED": PRIMER_MAX_NS_ACCEPTED,
    "PRIMER_EXPLAIN_FLAG": PRIMER_EXPLAIN_FLAG
}

**PRIMER_MAX_NS_ACCEPTED = 0**:  
To avoid picking primers containing ambigous bases, **differeces between Sanger sequence and reference CDS (indels and mismatches in the alignment) are substitued to 'N'**, then, the parameter `PRIMER_MAX_NS_ACCEPTED` were set to `0` to skip those bases.


In [30]:
# write params to 'config'
# config = "./configs/primer3.config"
open(config, "w").close()
with open(config, "a") as handle_out:
    handle_out.write("Primer3 File - http://primer3.org\n")
    handle_out.write("P3_FILE_TYPE=settings\n\n")
    for k, v in GENERAL.items():
        handle_out.write(f"{k}={v}\n")
    handle_out.write("=\n")

Primer3 per-read setting

In [4]:
PRIMER_PICK_LEFT_PRIMER: bool # == 1 for forward records
PRIMER_PICK_RIGHT_PRIMER: bool # == 1 for reverse records
PRIMER_EXPLAIN_FLAG: bool = 1 # 0 or 1

Execute Primer3

In [21]:
from subprocess import run

# Primer3 location
Primer3 = "./tools/primer3_core"

# I/O
test_input = "./test/test_primer3/example.in"
test_output = "./test/test_primer3/example.out"

proc = run([
    Primer3,
    f"--output={test_output}",
    f"--p3_settings_file={config}",
    test_input
    ],
    capture_output=True
    )
proc

CompletedProcess(args=['./tools/primer3_core', '--output=./test/test_primer3/example.out', '--p3_settings_file=./configs/primer3.config', './test/test_primer3/example.in'], returncode=0, stdout=b'', stderr=b'')

### Get re-seq name list (20220325)

In [3]:
import pandas as pd

df = pd.read_csv("output_classifier/summary.tsv", sep="\t")

with open("sequencing_request/20220325_to_be_sequenced.csv", "a") as fout:
    # iter over each row in `df`
    for row in df.iterrows():
        rname, fwd, rev, ovlp = row[1]
        if not fwd or not rev:
            req = "ADF" if not fwd else "ADR"
            fout.write(f"{rname},{req}\n")

### Visualing read length distribution after trimming with 'fastp'

In [ ]:
from Bio import SeqIO
read_len = []

for i in SeqIO.parse(open("raw_fastq/forward.fq", "r"), "fastq"):
    read_len.append(i.__len__())

for i in SeqIO.parse(open("raw_fastq/reverse.fq", "r"), "fastq"):
    read_len.append(i.__len__())

# create histogram
def hist(
    x:list, 
    interval:tuple, 
    binwidth:float,
    title:str="",
    outfile:str=None
    ) -> None:
    
    start, end = interval
    # initiate `hist_dict`
    hist_dict = {}
    k = start
    while k <= end:
        left = k
        right = k + binwidth
        key = f"{k:.1f}"
        hist_dict[key] = 0
        k += binwidth

    for i in x:
        bins = f"{float(i) // binwidth * binwidth:.1f}"
        hist_dict[bins] += 1
    
    # output
    if outfile == None:
        print(f"{title}")
        print(f"Histogram:")
        k = start
        while k <= end:
            left = k
            right = k + binwidth
            right = right if right <= end else left
            key = f"{k:.1f}"
            print(f"\t{left:.1f}-{right:.1f}: {hist_dict[key]}")
            k += binwidth
        print("\nParams:")
        print(f"\t-interval: ({start}, {end})")
        print(f"\t-binwidth: {binwidth}\n")
    else:
        with open(outfile, "a") as fout:
            fout.write(f"{title}\n")
            fout.write(f"Histogram:\n")
            k = start
            while k <= end:
                left = k
                right = k + binwidth
                right = right if right <= end else left
                key = f"{k:.1f}"
                fout.write(f"\t{left:.1f}-{right:.1f}: {hist_dict[key]}\n")
                k += binwidth
            fout.write("\nParams:\n")
            fout.write(f"\t-interval: ({start}, {end})\n")
            fout.write(f"\t-binwidth: {binwidth}\n\n")
    return

hist(read_len, (0, 1000), 50)